In [1]:

from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

#Loading the dataset from hugging face repository
1. train set: used for training the model
2. validation: used for validation set
3. test_dataset = used for testing the dataset and checking the perfomance
the data splitted into 70% trainingset 30% for testing and validation and further the 30% splitted into 50% test_set and 50%  validation_set


In [7]:
from datasets import load_dataset

# Load the 'Tvsybkzkmapab/Amharic_ad_generation' dataset
dataset = load_dataset('Tvsybkzkmapab/Amharic_ad_generation')

# Access the 'train', 'validation', and 'test' splits
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

# Print the first few samples from each split
print("\nTraining Dataset:")
print(train_dataset[:5])

print("\nValidation Dataset:")
print(validation_dataset[:5])

print("\nTest Dataset:")
print(test_dataset[:5])


Training Dataset:
{'text': ['ADVERTISMENT አፊያ ሙሀመድ ከፍተኛ የባህል ሕክምና እና ዘመናዊ የዋግምት አገልግሎት የምንሰጣቸው የባህል ህክምናዎች ➢ ለውጭና ለውስጥ ኪንታሮት ➢ ለማድያት ➢ ለሱኳር በሽታ ➢ለጉበት(ለወፍ በሽታ) ➢ለጨጎራ ህመም ➢ለስፈተወሲብ ➢ለደም ግፊት ➢ለአስም ወይም ሳይነስ ➢ለሚጥል በሺታ ➢ ለእሪህና ቁርጥማት ➢ለራስ ህመም (ማይግሪን) ➢ለቺፌ ና ለጭርት ➢ለቋቁቻና ፎረፎር ➢ለእጢና ለእባጭ ➢ለወገብ ህመም ➢ለመካንነት ለወድም ለሴትም ➢ለጆሮና ለአይን ህመም ➢ለሆድ ህመም ➢ዘመናዊ የዋግምት አገልግሎት በተቋማችን እንሰጣለን። 👉ከኢትዮጵያ ባህላዊ ህክምና አዋቂዎች ማህበር በዘርፉ ህጋዊ የባህል ህክምና ፍቃድ ያለን ነን። አድራሻ:አዲስ አበባ አየር ጤና ስልክ ቁጥር 📲0927506650 📲0987133734 📲0939605455 ቴሌግራም ቻናላችን ntvE5NmM0', 'ከአዝናኝ ጨዋታዎች እና ሽልማቶች ጋር በያላችሁበት እየደረስን ነው፡፡ ዕድሉ አያምልጣችሁ፤ አረንጓዴው የሳፋሪኮም አውቶብሳችን ባለበት በኩል ስታልፉ ተቀላቀሉን በአብሮነት ቆንጆ ጊዜ እናሳልፍ!\n\n#Gursha\n#SafaricomEthiopia #FurtherAheadTogether', 'ይነበብ! ቴዲ አፍሮ ከአሜሪካ መልክት ለኢትዮጵያውያን አስተላላፊ። ዶክተር አብይን ሀገር ውስጥ ሆኜ ባለመደገፌ በተሰምቶኛል።', 'በሳፋሪኮም ምቹ የድምጽ ፣ የፅሁፍ መልዕክት እና ፈጣን 4G የኢንተርኔት ድህረ ክፍያ ጥቅሎች ያለሃሳብ እንስራ ፤ ንግዳችንን እናሳድግ!\n\nለአገልግሎቱ ለመመዝገብ የሳፋሪኮምን ድህረ - ገፅ እንጎብኝ፣ ወደ 0700 755 755 or 0700 700 755 እንደውል ወይም ወደ enterprisesupport@safaricom.et ኢሜይል እንላክ።\n\n#Safarico

In [4]:
import sentencepiece as spm
import torch

# Load your SentencePiece tokenizer
sp = spm.SentencePieceProcessor(model_file="tokenizer.model")

def tokenize(prompt):
    # Tokenize the text using SentencePiece
    tokenized_sequence = sp.EncodeAsIds(prompt)

    # Padding and attention mask handling (adjust based on your tokenizer output)
    if len(tokenized_sequence) < 512:
        padding_length = 512 - len(tokenized_sequence)
        padded_sequence = tokenized_sequence + [0] * padding_length
        attention_mask = [1] * len(tokenized_sequence) + [0] * padding_length
    else:
        padded_sequence = tokenized_sequence[:512]
        attention_mask = [1] * 512

    # Create labels (adjust if needed)
    labels = padded_sequence.copy()

    # Convert lists to PyTorch tensors
    input_ids = torch.tensor(padded_sequence)
    attention_mask = torch.tensor(attention_mask)
    labels = torch.tensor(labels)

    # Combine data into result dict
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [12]:
# Model preparation and text generation
eval_prompt = "This is a sample evaluationprompt for the model."
model_input = tokenize(eval_prompt)  # Tokenize and create tensors
model_input = model_input.to("cuda")  # Move tensors to GPU if available

model.eval()  # Set model to evaluation mode

with torch.no_grad():
    generated_text = model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0]
    decoded_text = tokenize.decode(generated_text, skip_special_tokens=True)
    print(decoded_text)  # Print the generated text

NameError: name 'eval_prompt' is not defined